---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [2]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [3]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [4]:
def answer_one():
    return df[df['Gold'] == max(df['Gold'])].index[0]

answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [5]:
def answer_two():
    df['Difference'] = abs(df['Gold']-df['Gold.1'])
    return df[df['Difference'] == max(df['Difference'])].index[0]
    
answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [9]:
def answer_three():
    df_OneGold = df[(df['Gold'] > 0) & (df['Gold.1'] > 0)]
    df_OneGold['RelDiff'] = abs(df_OneGold['Gold']-df_OneGold['Gold.1'])/df_OneGold['Gold.2']
    return df_OneGold[df_OneGold['RelDiff'] == max(df_OneGold['RelDiff'])].index[0]
    
#answer_three()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [10]:
def answer_four():
    Points = df['Gold.2']*3 + df['Silver.2']*2 + df['Bronze.2']*1
    return Points

#answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [4]:
import pandas as pd
census_df = pd.read_csv('census.csv')
#census_df.head()

In [22]:
def answer_five():
    df = census_df[census_df['SUMLEV'] == 50]
    maxCounty = 0
    for item in df['STNAME'].unique():
        if len(df[df['STNAME'] == item]['COUNTY']) > maxCounty:
            maxCounty = len(df[df['STNAME'] == item]['COUNTY'])
            stateName = item
        #print(len(df[df['STNAME'] == item]['COUNTY']))
        #print(item)
    return stateName

def answer_five_alternative():
    df = census_df[census_df['SUMLEV'] == 50]
    x = df.groupby('STNAME')['SUMLEV'].count() #.count()['SUMLEV']
    return x[x == x.max()].index[0]
    #ans = x.idxmax()

answer_five_alternative()

'Texas'

### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [66]:
def answer_six():
    df=census_df[census_df['SUMLEV'] == 50]
    #df.head()
    ANSWER6 = pd.DataFrame()
    for item in df['STNAME'].unique():
        #find the population of the most populous counties in the state
        temp = df[df['STNAME'] == item]['CENSUS2010POP'].order(ascending=False)[:3]
        tempdata = pd.DataFrame({'Population' :  [sum(temp)]}, index=[item]) 
        ANSWER6 = ANSWER6.append(tempdata)

    return ANSWER6.sort(columns = 'Population', ascending=False).index[0:3].tolist()


def answer_six_alternative():
    df = census_df[census_df['SUMLEV'] == 50]
    x = pd.DataFrame(df.groupby('STNAME')['CENSUS2010POP'].nlargest(3))#.sum('STNAME')
    x = x.reset_index()
    x_prime = x.groupby('STNAME')['CENSUS2010POP'].sum() 
    return x_prime.nlargest(3).index.tolist()


answer_six_alternative()

['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [64]:
def answer_seven():
    df=census_df[census_df['SUMLEV'] == 50]
    #df.head()
    YEARS = ['POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012',
             'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']
    deltaPop = 0
    bigChangeCounty = None
    for countyCounter in range(0,len(df['COUNTY'])):  #len(df['COUNTY'])
        #find the biggest change in county population over the years and 
        #compare with the previously chosen biggest change
        for year1 in YEARS:
            for year2 in YEARS:
                if year1 != year2:
                    tempdiff = abs(df[year1].iloc[countyCounter] - df[year2].iloc[countyCounter])
                    if deltaPop <tempdiff:
                        deltaPop = tempdiff
                        bigChangeCounty = df['CTYNAME'].iloc[countyCounter]
                                          #df['STNAME'].iloc[countyCounter] + str(df['COUNTY'].iloc[countyCounter])
                    
    
    print(bigChangeCounty)
    print(deltaPop)
    return bigChangeCounty

def answer_seven_alternative():
    df = census_df[census_df['SUMLEV'] == 50]
    YEARS = ['POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012',
             'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']
    df['max'] = df[YEARS].max(axis=1)
    df['min'] = df[YEARS].min(axis=1)
    df['diff'] = df['max'] - df['min']

    return df[df['diff'] == df['diff'].max()]['CTYNAME'].values.tolist()#iloc[0]

answer_seven_alternative()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

['Harris County']

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [3]:
def answer_eight():
    
    df = census_df[census_df['SUMLEV'] == 50]
    ans = df[ ((df['REGION']==1)|(df['REGION']==2)) & (df['CTYNAME']=='Washington County') 
             & (df['POPESTIMATE2015']>df['POPESTIMATE2014'])][['STNAME','CTYNAME']]
    
    return ans

answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
